In [1]:
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))


In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from Camera import Camera
import numpy as np
from Frame import Frame


import matplotlib.pyplot as plt
from GenerateGsInput import GenerateGsInput
import Plotters
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}


In [63]:
# initilize objects
frames = list(range(900,930,1))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}

In [55]:
frames.keys()

dict_keys(['P325CAM1.jpg', 'P325CAM2.jpg', 'P325CAM3.jpg', 'P325CAM4.jpg', 'P326CAM1.jpg', 'P326CAM2.jpg', 'P326CAM3.jpg', 'P326CAM4.jpg', 'P327CAM1.jpg', 'P327CAM2.jpg', 'P327CAM3.jpg', 'P327CAM4.jpg', 'P328CAM1.jpg', 'P328CAM2.jpg', 'P328CAM3.jpg', 'P328CAM4.jpg', 'P329CAM1.jpg', 'P329CAM2.jpg', 'P329CAM3.jpg', 'P329CAM4.jpg', 'P330CAM1.jpg', 'P330CAM2.jpg', 'P330CAM3.jpg', 'P330CAM4.jpg', 'P331CAM1.jpg', 'P331CAM2.jpg', 'P331CAM3.jpg', 'P331CAM4.jpg', 'P332CAM1.jpg', 'P332CAM2.jpg', 'P332CAM3.jpg', 'P332CAM4.jpg', 'P333CAM1.jpg', 'P333CAM2.jpg', 'P333CAM3.jpg', 'P333CAM4.jpg', 'P334CAM1.jpg', 'P334CAM2.jpg', 'P334CAM3.jpg', 'P334CAM4.jpg'])

In [68]:
# plots
# camera ptcloud
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[50]
xyz = np.array(frames[im_name].points_in_idx[['X','Y','Z']])
Plotters.scatter3d(fig,xyz,color_pts[0],3)
fig.show()

In [8]:
ref_image = frames['P1407CAM4.jpg'].croped_image
im_name = list(frames.keys())[0]
from PIL import Image

# frames['P1407CAM2.jpg'].match_hist(ref_image)
# frames['P1407CAM3.jpg'].match_hist(ref_image)
# frames['P1407CAM1.jpg'].match_hist(ref_image)
arr_im = np.array(frames['P1407CAM4.jpg'].croped_image)

arr_im[arr_im > 0] = np.array(arr_im[arr_im > 0]) - 30
subs = [47,35,35,0]
# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    im_name = list(frames.keys())[cam]
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    arr_im = np.array(frames[im_name].croped_image)
    arr_im[arr_im > 0] = np.array(arr_im[arr_im > 0]) - subs[cam]*0 + 20*0
    frames[im_name].croped_image = Image.fromarray(arr_im)
    axs[cam // 2,cam % 2].imshow(arr_im,'gray')


In [30]:
ggs.path

'G:/My Drive/Research/gs_data/mov19_2022_03_03/'

In [3]:
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
# map 3d voxels to 2d pixels    

[frames[im_name].map_3d_2d(croped_image = True, use_zbuff = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)
ggs = GenerateGsInput(path,frames)
frames_dict = {}
idx = 0
for frame in range(1407,1430):
    
    image_name,points_in_idx = Utils.define_frames([frame],points_3d)
    cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
    frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}
    [frames[im_name].map_3d_2d(croped_image = True, use_zbuff = True) for im_name in frames.keys()]
    voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)
    ggs = GenerateGsInput(path,frames)
    
    ggs.save_croped_images(croped_image = True)
    ggs.generate_camera_text(croped_image = True)
    ggs.generate_image_text()
    ggs.generate_points3d_text(voxel_dict,colors_dict)
    base_images = ggs.generate_base_image()
    cams = ggs.generate_cams(croped_image = True)
    xyz,rgb = ggs.generate_xyz_rgb(voxel_dict,colors_dict)
    frames_dict[frame] = [base_images,cams,[xyz,rgb]]
    idx += 1


Utils.pickle_file(frames_dict,f'{ggs.sparse_dir}/frames_zbuff.pkl' )


In [4]:
# plots
# camera ptcloud
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[0]
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3)
fig.show()

In [31]:
Utils.pickle_file(frames_dict,f'{ggs.sparse_dir}/frames.pkl' )


In [5]:
ggs.sparse_dir

'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03//input_data_for_gs/sparse/'

In [48]:
base_images[1]

{'id': 1,
 'qvec': array([ 0.0101965,  0.49418  , -0.4900953,  0.7179755]),
 'tvec': array([ 2.34052445e-06, -3.10337357e-03,  1.96041261e-01]),
 'camera_id': 1,
 'name': 'P1407CAM2.jpg',
 'xys': array([[ 25.,  60.],
        [ 26.,  61.],
        [ 27.,  61.],
        ...,
        [126., 110.],
        [127., 108.],
        [127., 109.]]),
 'point3D_ids': array([16908, 16912, 16904, ...,    -1,    -1,    -1])}

In [26]:
frames_num = list(range(1407,1430,1))
frames_pkl = {}
for frame in  frames_num:
    frames_pkl[frame] = [base_images[4*idx]




dict_keys([1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419])

In [15]:
import pickle
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/input_data_for_gs/sparse/frames.pkl'
with open(path, 'rb') as file:
    data = pickle.load(file)


In [23]:
import pickle
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/input_data_for_gs/sparse/base_images.pkl'
with open(path, 'rb') as file:
    data = pickle.load(file)

data[90]

{'id': 90,
 'qvec': array([ 0.0055668, -0.5137345, -0.4945976,  0.7010129]),
 'tvec': array([-0.00187092, -0.00162703,  0.19135575]),
 'camera_id': 90,
 'name': 'P1429CAM3.jpg',
 'xys': array([[ 41.,  70.],
        [ 42.,  66.],
        [ 42.,  67.],
        ...,
        [123.,  67.],
        [123.,  68.],
        [123.,  69.]]),
 'point3D_ids': array([1068, 3352, 3392, ...,   -1,   -1,   -1])}

In [ ]:


# import pickle
# with open(f'{ggs.sparse_dir}/base_images.pkl', 'wb') as f:
#     pickle.dump(base_images, f)

# Utils.pickle_file(base_images, f'{ggs.sparse_dir}/base_images.pkl')
# Utils.pickle_file(cams, f'{ggs.sparse_dir}/cams.pkl')
# Utils.pickle_file([xyz,rgb], f'{ggs.sparse_dir}/xyz_rgb.pkl')



In [12]:
base_images

{0: {'id': 0,
  'qvec': array([ 0.4014182, -0.7619356,  0.4480878, -0.2398643]),
  'tvec': array([0.00099953, 0.00096338, 0.27667279]),
  'camera_id': 0,
  'name': 'P1407CAM1.jpg',
  'xys': array([[ 63., 120.],
         [ 63., 121.],
         [ 63., 122.],
         ...,
         [104.,  87.],
         [104.,  88.],
         [105.,  80.]]),
  'point3D_ids': array([19611, 19641, 19669, ...,    -1,    -1,    -1])},
 1: {'id': 1,
  'qvec': array([ 0.0101965,  0.49418  , -0.4900953,  0.7179755]),
  'tvec': array([ 2.34052445e-06, -3.10337357e-03,  1.96041261e-01]),
  'camera_id': 1,
  'name': 'P1407CAM2.jpg',
  'xys': array([[ 25.,  60.],
         [ 26.,  61.],
         [ 27.,  61.],
         ...,
         [126., 110.],
         [127., 108.],
         [127., 109.]]),
  'point3D_ids': array([16908, 16912, 16904, ...,    -1,    -1,    -1])},
 2: {'id': 2,
  'qvec': array([ 0.0055668, -0.5137345, -0.4945976,  0.7010129]),
  'tvec': array([-0.00187092, -0.00162703,  0.19135575]),
  'camera_id':

In [5]:
im_name = list(frames.keys())[0]
fig = go.Figure()
Plotters.scatter3d(fig,frames[im_name].points_in_ew_frame,'red',3)
fig.show()


In [7]:
# plots
# camera ptcloud
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[0]
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3)
fig.show()



# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    homo_voxels_with_idx = frames[image].add_homo_coords(frames[image].voxels_with_idx[:,0:3])
    proj = frames[image].project_on_image(homo_voxels_with_idx,croped_camera_matrix = True)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 3,c = 'red')


In [13]:
from colmap_loader import read_extrinsics_text, read_intrinsics_text, qvec2rotmat, \
    read_extrinsics_binary, read_intrinsics_binary, read_points3D_binary, read_points3D_text


cameras_extrinsic_file = f'{ggs.sparse_dir}/images.txt'
cameras_intrinsic_file = f'{ggs.sparse_dir}/cameras.txt'
txt_path =f'{ggs.sparse_dir}/points3D.txt'

xyz, rgb, _ = read_points3D_text(txt_path)

cam_extrinsics = read_extrinsics_text(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_text(cameras_intrinsic_file)

from PIL import Image

image = Image.open(f'{ggs.image_dir}/{im_name}')
image.mode


'RGB'

In [15]:
frames.keys()

dict_keys(['P1407CAM1.jpg', 'P1407CAM2.jpg', 'P1407CAM3.jpg', 'P1407CAM4.jpg'])

In [16]:
import collections
Camera = collections.namedtuple("Camera", ["id", "model", "width", "height", "params"])

camera_id = frames['P1407CAM1.jpg'].camera_number
model = 'PINHOLE'
width = frames['P1407CAM1.jpg'].crop_size
height= frames['P1407CAM1.jpg'].crop_size
intrinsic = frames['P1407CAM1.jpg'].K_crop 


params = intrinsic[0,0], intrinsic[1,1], intrinsic[0,2], intrinsic[1,2]
cameras[camera_id] = Camera(id=camera_id, model=model,
                                            width=width, height=height,
                                            params=params)

xyz_voxel = np.vstack([voxel_dict[key][:3] for key in voxel_dict.keys()])
rgb_voxel = np.vstack([colors_dict[key][:3] for key in colors_dict.keys()])

BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)
    


In [28]:
import collections
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])


class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)
    
images = {}
xyz_voxel = np.vstack([voxel_dict[key][:3] for key in voxel_dict.keys()])
rgb_voxel = np.vstack([colors_dict[key][:3] for key in colors_dict.keys()])

frames['P1407CAM1.jpg'].pixel_with_idx[:,0:2]


point3D_ids = frames['P1407CAM1.jpg'].pixel_with_idx[:,2].astype(int)
xys = frames['P1407CAM1.jpg'].pixel_with_idx[:,0:2]
image_id = frames['P1407CAM1.jpg'].image_id
qvec = frames['P1407CAM1.jpg'].qvec
tvec = frames['P1407CAM1.jpg'].t.T[0]
camera_id = frames['P1407CAM1.jpg'].camera_number
image_name = frames['P1407CAM1.jpg'].image_name

images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)

In [29]:
images

{0: Image(id=0, qvec=array([ 0.4014182, -0.7619356,  0.4480878, -0.2398643]), tvec=array([0.00099953, 0.00096338, 0.27667279]), camera_id=0, name='P1407CAM1.jpg', xys=array([[ 63., 120.],
        [ 63., 121.],
        [ 63., 122.],
        ...,
        [104.,  87.],
        [104.,  88.],
        [105.,  80.]]), point3D_ids=array([19611, 19641, 19669, ...,    -1,    -1,    -1]))}

In [27]:
frames['P1408CAM1.jpg'].pixel_with_idx[:,0:2]

extrinsics = {}

point3D_ids = frames['P1408CAM1.jpg'].pixel_with_idx[:,2].astype(int)
xys = frames['P1408CAM1.jpg'].pixel_with_idx[:,0:2]
image_id = frames['P1408CAM1.jpg'].image_id
qvec = frames['P1408CAM1.jpg'].qvec
tvec = frames['P1408CAM1.jpg'].t.T[0]
camera_id = frames['P1408CAM1.jpg'].camera_number
image_name = frames['P1408CAM1.jpg'].image_name

images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)

NameError: name 'Image' is not defined

In [9]:
images

{0: Image(id=0, qvec=array([ 0.4014182, -0.7619356,  0.4480878, -0.2398643]), tvec=array([0.00099953, 0.00096338, 0.27667279]), camera_id=0, name='im_name.jpg', xys=array([[ 61., 121.],
        [ 61., 122.],
        [ 61., 123.],
        ...,
        [106.,  88.],
        [107.,  79.],
        [107.,  80.]]), point3D_ids=array([20219, 20229, 20240, ...,    -1,    -1,    -1]))}

In [26]:
import math

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])



def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)



def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def getProjectionMatrix1(znear, zfar, fovX, fovY,cx,cy,im_size):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    P[0, 2] = (right + left) / (right - left) + (2 * cx / im_size[0]) - 1
    P[1, 2] = (top + bottom) / (top - bottom) + (2 * cy / im_size[1]) - 1
    return P

def getProjectionMatrixShift(znear, zfar, focal_x, focal_y, cx, cy, width, height, fovX, fovY):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    # the origin at center of image plane
    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    # shift the frame window due to the non-zero principle point offsets
    offset_x = cx - (width/2)
    offset_x = (offset_x/focal_x)*znear
    offset_y = cy - (height/2)
    offset_y = (offset_y/focal_y)*znear

    top = top + offset_y
    left = left + offset_x
    right = right + offset_x
    bottom = bottom + offset_y

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    return P

def adjust_focal_lengths(fx, fy, cx, cy, image_width, image_height, znear):
    # Shifts from the image center
    shift_x = cx - image_width / 2
    shift_y = cy - image_height / 2
    
    # Adjust the focal lengths based on the shifts
    adjusted_fx = fx - (shift_x / znear)
    adjusted_fy = fy - (shift_y / znear)
    
    return adjusted_fx, adjusted_fy


def adjust_translation(t, cx, cy, image_width, image_height):
    # Adjust the translation based on the shift from center
    t_adjusted = np.copy(t)
    t_adjusted[0] += (cx - image_width / 2) * 1  # Adjust for x shift
    t_adjusted[1] += (cy - image_height / 2) * 1 # Adjust for y shift
    
    return t_adjusted


def getProjectionMatrix(znear, zfar, fovX, fovY):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    P[0, 2] = (right + left) / (right - left) 
    P[1, 2] = (top + bottom) / (top - bottom) 
    return P

image_name = 'P1407CAM2.jpg'



image_width = 160
image_height = 160
znear = 0.000000001
zfar = 100

focalx = frames[image_name].K_crop[0,0]
focaly = frames[image_name].K_crop[1,1]
cx = frames[image_name].K_crop[0, 2]  # Updated cx for cropped image
cy = frames[image_name].K_crop[1, 2]  # Updated cy for cropped image


R = qvec2rotmat(frames[image_name].qvec).T
# R = frames[image_name].R
T = frames[image_name].t.T

im_size = [160,160]

# focalx,focaly = adjust_focal_lengths(focalx, focaly, cx, cy, image_width, image_height, znear)
fovX = focal2fov(focalx, image_width)
fovY = focal2fov(focaly, image_height)

world_view_transform = getWorld2View2(R, T)
# projection_matrix = getProjectionMatrix(znear, zfar, fovX, fovY)
projection_matrix = getProjectionMatrix1(znear, zfar, fovX, fovY,cx,cy,im_size)

projection_matrix = getProjectionMatrixShift(znear, zfar, focalx, focaly, cx, cy, image_width, image_height, fovX, fovY)
full_proj_transform = np.matmul(projection_matrix,world_view_transform)  # Shape: (1, N, K)
# full_proj_transform = np.dot(projection_matrix,world_view_transform)

xyz_homo  = np.column_stack((frames[image_name].voxels_with_idx[:,0:3],np.ones((frames[image_name].voxels_with_idx.shape[0],1))))

p_proj = np.matmul(full_proj_transform,xyz_homo.T).T

p_view = np.matmul(world_view_transform,xyz_homo.T).T


p_proj = p_proj[:,0:3]/p_proj[:,3:]

def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel_x =proj_screen(p_proj[:,0],image_width)
pixel_y =proj_screen(p_proj[:,1],image_height)
plt.figure()
plt.imshow(frames[image_name].croped_image)

plt.scatter(pixel_x,pixel_y)



In [10]:
import numpy as np

# Parameters
W, H = 1024, 768  # Image dimensions
block_x, block_y = 16, 16  # Block dimensions

# Create a sample 2D image array filled with zeros
image = np.zeros((H, W), dtype=np.float32)

# Initialize a "grid" that covers the image dimensions
grid_x = (W + block_x - 1) // block_x
grid_y = (H + block_y - 1) // block_y

def process_pixel(x, y):
    """
    Simulate some processing on a pixel at (x, y).
    Here, we're just assigning the pixel the value of (x + y) for demonstration.
    """
    return x + y

# Loop over each block in the "grid"
for block_idx_y in range(grid_y):
    for block_idx_x in range(grid_x):
        
        # Loop over each "thread" (pixel) within the current block
        for thread_idx_y in range(block_y):
            for thread_idx_x in range(block_x):
                
                # Calculate the pixel (x, y) coordinates in the image
                x = block_idx_x * block_x + thread_idx_x
                y = block_idx_y * block_y + thread_idx_y

                # Check if the pixel is within image bounds
                if x < W and y < H:
                    # Process the pixel and store the result in the image array
                    image[y, x] = process_pixel(x, y)

# Display a portion of the processed image for verification


grid_x


64

In [72]:
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO

def process_ply_to_splat(ply_file_path):
    pos = []
    color = []

    plydata = PlyData.read(ply_file_path)
    vert = plydata["vertex"]
    sorted_indices = np.argsort(
        -np.exp(vert["scale_0"] + vert["scale_1"] + vert["scale_2"])
        / (1 + np.exp(-vert["opacity"]))
    )
    buffer = BytesIO()
    for idx in sorted_indices:
        v = plydata["vertex"][idx]
        pos.append(np.array([v["x"], v["y"], v["z"]], dtype=np.float32))
        scales = np.exp(
            np.array(
                [v["scale_0"], v["scale_1"], v["scale_2"]],
                dtype=np.float32,
            )
        )
        rot = np.array(
            [v["rot_0"], v["rot_1"], v["rot_2"], v["rot_3"]],
            dtype=np.float32,
        )
        SH_C0 = 0.28209479177387814
        color.append(np.array(
            [
                0.5 + SH_C0 * v["f_dc_0"],
                0.5 + SH_C0 * v["f_dc_1"],
                0.5 + SH_C0 * v["f_dc_2"],
                1 / (1 + np.exp(-v["opacity"])),
            ]
        ))
       
        

    return pos,scales,color


input_file = "D:/Documents/gray_dense_grad/frame1407/output/position_lr_init_1e-05_scaling_lr0.005_dens_iter100_featur0.0025/point_cloud/iteration_15000/point_cloud.ply"
splat_data = process_ply_to_splat(input_file)

In [85]:
input_file.split('/')[-1]

'point_cloud.ply'

In [73]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

pio.renderers.default = 'browser'

# Assuming 'splat_data' is predefined with your data
pts = np.vstack(splat_data[0])
clr = np.vstack(splat_data[2])

pts2del = np.where((clr[:, 2] >0.05) & (clr[:, 3] < 0.01) & (clr[:, 2] < 1))
# pts2del = np.where((clr[:, 2] >0.05) & (clr[:, 2] < 1))

colors = clr[pts2del[0], 1]
x = pts[pts2del[0], 0]
y = pts[pts2del[0], 1]
z = pts[pts2del[0], 2]
opa = clr[pts2del[0], 3]


# x = pts[:, 0]
# y = pts[:, 1]
# z = pts[:, 2]
# opa = clr[:, 3]

# Create the 3D scatter plot with hover template for color value
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=clr,  # Color for each point
        colorscale='Viridis',  # Choose a colorscale
        opacity=0.5,  # Use the opacity per point
        colorbar=dict(title="Colorbar")  # Optional colorbar
    ),
    hovertemplate='Color: %{marker.color:.4f}<extra></extra>',  # Show color as float
)])

# Update layout
fig.update_layout(scene=dict(
                    xaxis_title='X Axis',
                    yaxis_title='Y Axis',
                    zaxis_title='Z Axis'),
                  title="3D Scatter Plot with Color Hover as Float")

# Show the plot
# for cam in range(4):
#     image = list(frames.keys())[cam]

#     Plotters.scatter3d(fig,frames[image].voxels_with_idx,'red',3,opa = 0.2)

# fig.show()


In [51]:
color[3]

array([0.02062435, 0.02727893, 0.13483195, ..., 0.02147318, 0.00367921,
       0.9630995 ], dtype=float32)

In [83]:
vertices = plydata["vertex"]
xyz = np.column_stack((vertices["x"], vertices["y"], vertices["z"]))
scale = np.exp(np.column_stack(([vertices["scale_0"], vertices["scale_1"], vertices["scale_2"]])))
xyz.shape
scale.shape
SH_C0 = 0.28209479177387814
color = np.column_stack([
0.5 + SH_C0 * vertices["f_dc_0"],
0.5 + SH_C0 * vertices["f_dc_1"],
0.5 + SH_C0 * vertices["f_dc_2"],
1 / (1 + np.exp(-vertices["opacity"])),])
color.shape


(21627, 4)

In [75]:
from plyfile import PlyData, PlyElement
import numpy as np

# Load the .ply file
plydata = PlyData.read(input_file)

vertices = plydata['vertex']
# Access specific properties such as x, y, z
x = np.array(vertices['x'])
y = np.array(vertices['y'])
z = np.array(vertices['z'])
SH_C0 = 0.28209479177387814
color = np.array(
    [
        0.5 + SH_C0 * vertices["f_dc_0"],
        0.5 + SH_C0 * vertices["f_dc_1"],
        0.5 + SH_C0 * vertices["f_dc_2"],
        1 / (1 + np.exp(-vertices["opacity"])),
    ]
)



# Update the rest of the properties as well
filtered_vertices = vertices[(color[2] >= 0.1) & (color[2] < 1)]

# Convert filtered vertices to a structured array
# new_vertex_data = [(filtered_vertices[i][name] for name in vertices) for i in range(len(filtered_vertices))]
# new_vertex_data = np.array(new_vertex_data, dtype=vertices.dtype)

# Create a new PlyElement with the filtered vertices
filtered_element = PlyElement.describe(filtered_vertices, 'vertex')

# Save the filtered points back to a new .ply file

PlyData([filtered_element]).write(input_file.split('.ply')[0]+'_nob.ply')


In [23]:
len(filtered_element)

4919

In [38]:
len(filtered_vertices)

15747

In [17]:
ply_file_path  ="D:/Documents/point_cloud54354.ply"
plydata = PlyData.read(ply_file_path)
vert = plydata["vertex"]
plydata["vertex"]['f_dc_0']


xyz = np.vstack((plydata["vertex"]['x'],plydata["vertex"]['y'],plydata["vertex"]['z']))
normals = np.zeros_like(xyz)
f_dc = np.vstack((plydata["vertex"]['f_dc_0'],plydata["vertex"]['f_dc_1'],plydata["vertex"]['f_dc_2']))


from plyfile import PlyData, PlyElement
import os
def save_ply(self, path):
    mkdir_p(os.path.dirname(path))

    xyz = np.vstack((plydata["vertex"]['x'],plydata["vertex"]['y'],plydata["vertex"]['z']))
    normals = np.zeros_like(xyz)
    f_dc = self._features_dc.detach().transpose(1, 2).flatten(start_dim=1).contiguous().cpu().numpy()
    f_rest = self._features_rest.detach().transpose(1, 2).flatten(start_dim=1).contiguous().cpu().numpy()
    opacities = self._opacity.detach().cpu().numpy()
    scale = self._scaling.detach().cpu().numpy()
    rotation = self._rotation.detach().cpu().numpy()

    dtype_full = [(attribute, 'f4') for attribute in self.construct_list_of_attributes()]

    elements = np.empty(xyz.shape[0], dtype=dtype_full)
    attributes = np.concatenate((plydata[''], normals, f_dc, f_rest, opacities, scale, rotation), axis=1)
    elements[:] = list(map(tuple, attributes))
    el = PlyElement.describe(elements, 'vertex')
    PlyData([el]).write(path)

In [91]:
plt.plot(np.sort(x))


dist = np.linalg.norm(pts[pts2del[0],:]-frames[image].voxels_with_idx[:,0:3])

ValueError: operands could not be broadcast together with shapes (7374,3) (2540,3) 

In [85]:
plt.plot(frames[image].voxels_with_idx[:,0])

In [72]:
pts =np.vstack(splat_data[0])

color_pts = ['red','green','blue','black']
fig = go.Figure()
Plotters.scatter3d(fig,pts[pts2del[0],:],color_pts[0],3)
fig.show()


In [5]:


mul1 = (proj[0,0] * m_x + proj[1,0] * m_y + proj[2,0] * m_z + proj[3,0])*m_w
mul2 = (proj[0,1] * m_x + proj[1,1] * m_y + proj[2,1] * m_z + proj[3,1])*m_w

plt.scatter(mul1,mul2)

NameError: name 'proj' is not defined

In [6]:
	dL_dmean.x = (proj[0] * m_w - proj[3] * mul1) * dL_dmean2D[idx].x + (proj[1] * m_w - proj[3] * mul2) * dL_dmean2D[idx].y;
	dL_dmean.y = (proj[4] * m_w - proj[7] * mul1) * dL_dmean2D[idx].x + (proj[5] * m_w - proj[7] * mul2) * dL_dmean2D[idx].y;

NameError: name 'proj' is not defined

In [81]:
frames['P550CAM2.jpg'].world_to_cam.T.shape

(4, 3)

In [7]:
frames['P550CAM3.jpg'].t

NameError: name 'frames' is not defined

In [8]:
wakk

NameError: name 'wakk' is not defined

array([[-0.47209181,  0.5003788 , -0.72577571,  0.        ],
       [ 0.51598852, -0.51068448, -0.68771885,  0.        ],
       [-0.71476232, -0.69915837, -0.01709975,  0.        ],
       [-0.00187092, -0.00162703,  0.19135575,  1.        ]])

In [9]:
args['arr_8']

NameError: name 'args' is not defined

In [43]:
args['arr_9']

array([[ 2.8788988e+01,  2.6489201e+01,  7.1968478e-01,  7.1961278e-01],
       [ 2.8094706e+01,  2.9289333e+01, -6.9374442e-01, -6.9367504e-01],
       [-3.9387680e+01,  4.0252998e+01,  3.1188568e-02,  3.1185448e-02],
       [-1.3176772e-04,  1.7499962e-01,  1.8605986e-01,  1.9604126e-01]],
      dtype=float32)

In [22]:
args['arr_1']

fig = go.Figure()

Plotters.scatter3d(fig,frames['P550CAM1.jpg'].voxels_with_idx,'blue',3)
Plotters.scatter3d(fig,args['arr_1'],'red',3)

fig.show()

In [23]:
image_rgb = image.convert('RGB')
image_rgb.save('wakk.jpg')


In [24]:
image.size

pt = 'D:\Documents\south-building\images\P1180141.jpg'
image = Image.open('wakk.jpg')
image.mode

'RGB'